# Projet 7 - Elaborer un modele de scoring

## Feature Engineering

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_theme(palette="Set1")

### Chargement dataset

In [3]:
test_df = pd.read_csv("../Sources/application_test.csv")
train_df = pd.read_csv("../Sources/application_train.csv")
prev_df = pd.read_csv("../Sources/previous_application.csv")
br_df = pd.read_csv("../Sources/bureau.csv")
brb_df = pd.read_csv("../Sources/bureau_balance.csv")
pos_df = pd.read_csv("../Sources/POS_CASH_balance.csv")
cc_df = pd.read_csv("../Sources/credit_card_balance.csv")
inst_df = pd.read_csv("../Sources/installments_payments.csv")

#### Pour toute variable catégorielle (dtype == object) comportant 2 catégories uniques, nous utiliserons le label encoding, et pour toute variable catégorielle comportant plus de 2 catégories uniques, nous utiliserons le one hot encoder.

#### Pour le label encoding, nous utilisons Scikit-Learn LabelEncoder et pour le one hot encoder, la fonction pandas get_dummies(df)

In [4]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train_df:
    if train_df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train_df[col].unique())) <= 2:
            # Train on the training data
            le.fit(train_df[col])
            # Transform both training and testing data
            train_df[col] = le.transform(train_df[col])
            test_df[col] = le.transform(test_df[col])

            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [5]:
# one-hot encoding of categorical variables
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

print('Training Features shape: ', train_df.shape)
print('Testing Features shape: ', test_df.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


In [6]:
# Function to one hot encode categorical variables
def one_hot_encoding(data, nan_as_category=True):
    # Get columns
    original_cols = data.columns.to_list()
    categorical_cols = data.select_dtypes(object).columns.to_list()

    # One hot encode
    data = pd.get_dummies(data, columns=categorical_cols, dummy_na=nan_as_category)

    # Compute new columns
    new_columns = [col for col in data.columns if col not in original_cols]

    return data, new_columns

### Alignement des données d'entraînement et de test

#### Les données d'entraînement et de test doivent comporter les mêmes variables. Le one hot encoding a créé davantage de colonnes dans les données d'entraînement, car certaines variables catégorielles comportaient des catégories non représentées dans les données de test.

#### Pour supprimer les colonnes des données d'entraînement qui ne figurent pas dans les données de test, nous devons aligner les dataframes. Nous commençons par extraire la colonne cible des données d'entraînement (car elle ne figure pas dans les données de test, mais nous devons conserver cette information). 

#### Lorsque nous effectuons l'alignement, nous devons veiller à définir axis = 1 afin d'aligner les dataframes en fonction des colonnes et non des lignes

In [7]:
train_labels = train_df['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
train_df, test_df = train_df.align(test_df, join='inner', axis=1)

# Add the target back in
train_df['TARGET'] = train_labels

print('Training Features shape: ', train_df.shape)
print('Testing Features shape: ', test_df.shape)

Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)


### Application Test

In [8]:
# Define usefull variables
TARGET = 'TARGET' 
ID = 'SK_ID_CURR'

In [9]:
# Display dataframes
display("Application test:", test_df.head())
display("Application train:", train_df.head())

'Application test:'

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100001,0,0,1,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,False,False,False,False,False,False,True,False,True,False
1,100005,0,0,1,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,False,False,False,False,False,False,False,False,False,False
2,100013,0,1,1,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,False,False,False,False,False,False,False,False,False,False
3,100028,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,False,False,False,False,False,True,False,False,True,False
4,100038,0,1,0,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,False,False,False,False,False,False,False,False,False,False


'Application train:'

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,False,False,False,False,False,True,False,True,False,1
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,True,False,False,False,False,False,False,True,False,0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,False,False,False,False,False,False,False,False,False,0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,False,False,False,False,False,False,False,False,False,0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,False,False,False,False,False,False,False,False,False,0


In [10]:
# Store ids
test_ids = test_df[ID].to_list()

# Join dataframes
train_df = pd.concat([train_df, test_df])
display(train_df)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,False,False,False,False,False,True,False,True,False,1.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,True,False,False,False,False,False,False,True,False,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,False,False,False,False,False,False,False,False,False,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,False,False,False,False,False,False,False,False,False,0.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,False,False,False,False,False,False,False,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,0,1,0,121500.0,412560.0,17473.5,270000.0,0.002042,...,False,False,False,False,False,False,False,False,False,NaN
48740,456222,0,0,0,2,157500.0,622413.0,31909.5,495000.0,0.035792,...,False,False,False,False,False,False,False,False,False,NaN
48741,456223,0,1,1,1,202500.0,315000.0,33205.5,315000.0,0.026392,...,False,False,False,False,False,True,False,True,False,NaN
48742,456224,0,0,0,0,225000.0,450000.0,25128.0,450000.0,0.018850,...,False,False,False,False,True,False,False,True,False,NaN


### Application Train

In [11]:
train_df

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,False,False,False,False,False,True,False,True,False,1.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,True,False,False,False,False,False,False,True,False,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,False,False,False,False,False,False,False,False,False,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,False,False,False,False,False,False,False,False,False,0.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,False,False,False,False,False,False,False,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,0,1,0,121500.0,412560.0,17473.5,270000.0,0.002042,...,False,False,False,False,False,False,False,False,False,NaN
48740,456222,0,0,0,2,157500.0,622413.0,31909.5,495000.0,0.035792,...,False,False,False,False,False,False,False,False,False,NaN
48741,456223,0,1,1,1,202500.0,315000.0,33205.5,315000.0,0.026392,...,False,False,False,False,False,True,False,True,False,NaN
48742,456224,0,0,0,0,225000.0,450000.0,25128.0,450000.0,0.018850,...,False,False,False,False,True,False,False,True,False,NaN


In [12]:
train_df.columns.tolist()

['SK_ID_CURR',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NON

In [13]:
# Find code gender == xnaf entries
xnaf_idx = train_df[train_df['CODE_GENDER_F'] == 'XNA'].index

# Remove them
train_df = train_df.drop(xnaf_idx)
print("CODE_GENDER: Removed {} entries (XNAF)".format(len(xnaf_idx)))

CODE_GENDER: Removed 0 entries (XNAF)


In [14]:
# Find code gender == xnam entries
xnam_idx = train_df[train_df['CODE_GENDER_M'] == 'XNA'].index

# Remove them
train_df = train_df.drop(xnam_idx)
print("CODE_GENDER: Removed {} entries (XNAM)".format(len(xnam_idx)))

CODE_GENDER: Removed 0 entries (XNAM)


In [15]:
# Find anomalie value for days employed
anom_value = train_df['DAYS_EMPLOYED'].max()

# Replace by NaN
train_df = train_df.replace({anom_value: np.nan})
print("DAYS_EMPLOYED: replace {} by NaN.".format(anom_value))

DAYS_EMPLOYED: replace 365243 by NaN.


In [16]:
# Create the new features
train_df['DAYS_EMPLOYED_PERCENT'] = train_df['DAYS_EMPLOYED'] / train_df['DAYS_BIRTH']
train_df['INCOME_CREDIT_PERCENT'] = train_df['AMT_INCOME_TOTAL'] / train_df['AMT_CREDIT']
train_df['INCOME_PER_PERSON'] = train_df['AMT_INCOME_TOTAL'] / train_df['CNT_FAM_MEMBERS']
train_df['ANNUITY_INCOME_PERCENT'] = train_df['AMT_ANNUITY'] / train_df['AMT_INCOME_TOTAL']
train_df['PAYMENT_RATE'] = train_df['AMT_ANNUITY'] / train_df['AMT_CREDIT']

In [17]:
train_df.head(  )

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET,DAYS_EMPLOYED_PERCENT,INCOME_CREDIT_PERCENT,INCOME_PER_PERSON,ANNUITY_INCOME_PERCENT,PAYMENT_RATE
0,100002.0,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,True,False,True,False,1.0,0.067329,0.498036,202500.0,0.121978,0.060749
1,100003.0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,False,False,True,False,0.0,0.070862,0.208736,135000.0,0.132217,0.027598
2,100004.0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,False,False,False,False,0.0,0.011814,0.500000,67500.0,0.100000,0.050000
3,100006.0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,False,False,False,False,0.0,0.159905,0.431748,67500.0,0.219900,0.094941
4,100007.0,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,False,False,False,False,0.0,0.152418,0.236842,121500.0,0.179963,0.042623


In [18]:
# Prepare dataframe for merge
merged_df = train_df.copy()

### Bureau et bureau balance

* One hot encode categorical variables
* Merge bureau_balance to bureau by SK_ID_BUREAU
* Create one feature : BURO_COUNT (count of previous loans)
* Merge bureau to application_train by ID

In [19]:
# Display dataframes
display(br_df.head())
display(brb_df.head())

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [20]:
# One-hot encode bureau
br_df, br_encoded_cols = one_hot_encoding(br_df, nan_as_category=True)
display(br_df.head())

# One-hot encode bureau balance
brb_df, brb_encoded_cols = one_hot_encoding(brb_df, nan_as_category=True)
display(brb_df.head())

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,False,False,False,False,False,False,False,False,False,False
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,False,False,False,False,False,False,False,False,False,False
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,False,False,False,False,False,False,False,False,False,False
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,False,False,False,False,False,False,False,False,False,False


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,STATUS_nan
0,5715448,0,False,False,False,False,False,False,True,False,False
1,5715448,-1,False,False,False,False,False,False,True,False,False
2,5715448,-2,False,False,False,False,False,False,True,False,False
3,5715448,-3,False,False,False,False,False,False,True,False,False
4,5715448,-4,False,False,False,False,False,False,True,False,False


In [21]:
# Define aggregations
brb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}

# Add encoded columns aggregations
for col in brb_encoded_cols:
    brb_aggregations[col] = ['mean']
print(brb_aggregations)

{'MONTHS_BALANCE': ['min', 'max', 'size'], 'STATUS_0': ['mean'], 'STATUS_1': ['mean'], 'STATUS_2': ['mean'], 'STATUS_3': ['mean'], 'STATUS_4': ['mean'], 'STATUS_5': ['mean'], 'STATUS_C': ['mean'], 'STATUS_X': ['mean'], 'STATUS_nan': ['mean']}


In [22]:
# Function to flat index renaming
def rename_grouped_index(grouped_df, prefix=''):
    return pd.Index([prefix + elem[0] + "_" + elem[1].upper() for elem in grouped_df.columns.tolist()])

In [23]:
# Groupby bureau ID
brb_grouped = brb_df.groupby('SK_ID_BUREAU').agg(brb_aggregations)

# Rename columns
brb_grouped.columns = rename_grouped_index(brb_grouped)
display(brb_grouped.head())

,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_nan_MEAN
SK_ID_BUREAU,,,,,,,,,,,,
5001709,-96,0,97,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,0.0
5001710,-82,0,83,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,0.0
5001711,-3,0,4,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.0
5001712,-18,0,19,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,0.0
5001713,-21,0,22,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0


In [24]:
# Join the bureau balance grouped to bureau dataframe
br_df = br_df.join(brb_grouped, how='left', on='SK_ID_BUREAU')

# Remove bureau id
brb_df = brb_df.drop(['SK_ID_BUREAU'], axis=1)
display(br_df.head())

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_nan_MEAN
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Define bureau aggregations
br_aggregations = {'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
                   'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
                   'DAYS_CREDIT_UPDATE': ['mean'],
                   'CREDIT_DAY_OVERDUE': ['max', 'mean'],
                   'AMT_CREDIT_MAX_OVERDUE': ['mean'],
                   'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
                   'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
                   'AMT_CREDIT_SUM_OVERDUE': ['mean'],
                   'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
                   'AMT_ANNUITY': ['max', 'mean'],
                   'CNT_CREDIT_PROLONG': ['sum'],
                   'MONTHS_BALANCE_MIN': ['min'],
                   'MONTHS_BALANCE_MAX': ['max'],
                   'MONTHS_BALANCE_SIZE': ['mean', 'sum']}

# Add categorical aggregations
for col in br_encoded_cols:
    br_aggregations[col] = ['mean']
for col in brb_encoded_cols:
    br_aggregations[col + "_MEAN"] = ['mean']
# print(br_aggregations)

In [26]:
# Group by ID
br_grouped = br_df.groupby(ID).agg(br_aggregations)

# Rename columns
br_grouped.columns = rename_grouped_index(br_grouped, prefix='BURO_')

display(br_grouped.head())

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,BURO_CREDIT_TYPE_nan_MEAN,BURO_STATUS_0_MEAN_MEAN,BURO_STATUS_1_MEAN_MEAN,BURO_STATUS_2_MEAN_MEAN,BURO_STATUS_3_MEAN_MEAN,BURO_STATUS_4_MEAN_MEAN,BURO_STATUS_5_MEAN_MEAN,BURO_STATUS_C_MEAN_MEAN,BURO_STATUS_X_MEAN_MEAN,BURO_STATUS_nan_MEAN_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,-1329.0,1778.0,82.428571,-93.142857,0,0.0,...,0.0,0.336651,0.007519,0.0,0.0,0.0,0.0,0.441240,0.214590,0.0
100002,-1437,-103,-874.000000,186150.000000,-1072.0,780.0,-349.000000,-499.875000,0,0.0,...,0.0,0.406960,0.255682,0.0,0.0,0.0,0.0,0.175426,0.161932,0.0
100003,-2586,-606,-1400.750000,827783.583333,-2434.0,1216.0,-544.500000,-816.000000,0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,-1326,-408,-867.000000,421362.000000,-595.0,-382.0,-488.500000,-532.000000,0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-373,-62,-190.666667,26340.333333,-128.0,1324.0,439.333333,-54.333333,0,0.0,...,0.0,0.735043,0.000000,0.0,0.0,0.0,0.0,0.128205,0.136752,0.0


In [27]:
# Count previous loans count
br_grouped['BURO_COUNT'] = br_df.groupby(ID).size()

In [28]:
# Merge to application dataframe
merged_df = merged_df.join(br_grouped, how='left', on=ID)

### Previous Application

* Replace DAYS_... anomalie value by NaN
* Create one feature : APP_CREDIT_PERCENT (feature application credit percentage)
* One hot encode categorical variables
* Merge bureau to application_train by ID

In [29]:
# Display dataframe
display(prev_df.head())

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Display days variables
prev_days_cols = ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']
display(prev_df[prev_days_cols].describe())

# Find anomalie value for days employed
anom_value = prev_df['DAYS_FIRST_DRAWING'].max()

# Replace by NaN
prev_df[prev_days_cols] = prev_df[prev_days_cols].replace({anom_value: np.nan})
print("DAYS_...: replace {} by NaN.".format(anom_value))

,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
count,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000
mean,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838
std,88916.115833,72444.869708,106857.034789,149647.415123,153303.516729
min,-2922.000000,-2892.000000,-2801.000000,-2889.000000,-2874.000000
25%,365243.000000,-1628.000000,-1242.000000,-1314.000000,-1270.000000
50%,365243.000000,-831.000000,-361.000000,-537.000000,-499.000000
75%,365243.000000,-411.000000,129.000000,-74.000000,-44.000000
max,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000


DAYS_...: replace 365243.0 by NaN.


In [31]:
# Add feature application credit percentage
prev_df['APP_CREDIT_PERCENT'] = prev_df['AMT_APPLICATION'] / prev_df['AMT_CREDIT']

In [32]:
# Encode the categorical variables
prev_df, prev_encoded_cols = one_hot_encoding(prev_df, nan_as_category=True)
display(prev_df.head())

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,...,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_nan
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,1,0.0,...,False,False,False,False,False,True,False,False,False,False
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,1,NaN,...,False,False,False,False,False,False,False,False,False,False
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,11,1,NaN,...,False,False,False,False,False,False,False,False,False,False
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,7,1,NaN,...,True,False,False,False,False,False,False,False,False,False
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,9,1,NaN,...,False,False,False,False,False,False,False,False,False,False


In [33]:
# Numerical columns aggregations
prev_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERCENT': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum']}

# Add categorical columns aggregations
for col in prev_encoded_cols:
    prev_aggregations[col] = ['mean']

In [34]:
# Group by ID
prev_grouped = prev_df.groupby(ID).agg(prev_aggregations)

# Rename columns
prev_grouped.columns = rename_grouped_index(prev_grouped, prefix='PREV_')
display(prev_grouped.head())

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_APP_CREDIT_PERCENT_MIN,...,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN,PREV_PRODUCT_COMBINATION_POS household with interest_MEAN,PREV_PRODUCT_COMBINATION_POS household without interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,PREV_PRODUCT_COMBINATION_POS others without interest_MEAN,PREV_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,24835.5,24835.5,24835.50,23787.0,23787.0,23787.00,1.044079,...,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
100002,9251.775,9251.775,9251.775,179055.0,179055.0,179055.00,179055.0,179055.0,179055.00,1.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
100003,6737.310,98356.995,56553.990,68809.5,900000.0,435436.50,68053.5,1035882.0,484191.00,0.868825,...,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0
100004,5357.250,5357.250,5357.250,24282.0,24282.0,24282.00,20106.0,20106.0,20106.00,1.207699,...,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0
100005,4813.200,4813.200,4813.200,0.0,44617.5,22308.75,0.0,40153.5,20076.75,1.111173,...,0.0,0.000000,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.0


In [35]:
# Merge to application dataframe
merged_df = merged_df.join(prev_grouped, how='left', on=ID)

### POS cash balance

* One hot encode categorical variables
* Create one feature : POS_COUNT (pos cash accounts)
* Merge pos_cash_balance to application_train by ID

In [36]:
# Display dataframe
display(pos_df.head())

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [37]:
# One hot encode categorical columns
pos_df, pos_encoded_cols = one_hot_encoding(pos_df, nan_as_category=True)
display(pos_df.head())

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,NAME_CONTRACT_STATUS_nan
0,1803195,182943,-31,48.0,45.0,0,0,True,False,False,False,False,False,False,False,False,False
1,1715348,367990,-33,36.0,35.0,0,0,True,False,False,False,False,False,False,False,False,False
2,1784872,397406,-32,12.0,9.0,0,0,True,False,False,False,False,False,False,False,False,False
3,1903291,269225,-35,48.0,42.0,0,0,True,False,False,False,False,False,False,False,False,False
4,2341044,334279,-35,36.0,35.0,0,0,True,False,False,False,False,False,False,False,False,False


In [38]:
# Columns aggregations
pos_aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']}

# Add encoded columns aggregations
for cat in pos_encoded_cols:
    pos_aggregations[cat] = ['mean']
print(pos_aggregations)

{'MONTHS_BALANCE': ['max', 'mean', 'size'], 'SK_DPD': ['max', 'mean'], 'SK_DPD_DEF': ['max', 'mean'], 'NAME_CONTRACT_STATUS_Active': ['mean'], 'NAME_CONTRACT_STATUS_Amortized debt': ['mean'], 'NAME_CONTRACT_STATUS_Approved': ['mean'], 'NAME_CONTRACT_STATUS_Canceled': ['mean'], 'NAME_CONTRACT_STATUS_Completed': ['mean'], 'NAME_CONTRACT_STATUS_Demand': ['mean'], 'NAME_CONTRACT_STATUS_Returned to the store': ['mean'], 'NAME_CONTRACT_STATUS_Signed': ['mean'], 'NAME_CONTRACT_STATUS_XNA': ['mean'], 'NAME_CONTRACT_STATUS_nan': ['mean']}


In [39]:
# Group by ID
pos_grouped = pos_df.groupby(ID).agg(pos_aggregations)

# Rename columns
pos_grouped.columns = rename_grouped_index(pos_grouped, prefix='POS_')

display(pos_grouped.head())

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0


In [40]:
# Count pos cash accounts
pos_grouped['POS_COUNT'] = pos_df.groupby(ID).size()

In [41]:
# Merge to application dataframe
merged_df = merged_df.join(pos_grouped, how='left', on=ID)

### Installments payments

* Add 4 features : PAYMENT_PERCENT, PAYMENT_DIFF, DAYS_PAST_DUE and DAYS_BEFORE_DUE
* One hot encode categorical variables
* Add one feature : INST_COUNT (installments accounts)
* Merge installments_payments to application_train by ID

In [42]:
# Display dataframe
display(inst_df.head())

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [43]:
# Percentage and difference paid in each installment (amount paid and installment value)
inst_df['PAYMENT_PERCENT'] = inst_df['AMT_PAYMENT'] / inst_df['AMT_INSTALMENT']
inst_df['PAYMENT_DIFF'] = inst_df['AMT_INSTALMENT'] - inst_df['AMT_PAYMENT']

# Days past due and days before due (no negative values)
inst_df['DAYS_PAST_DUE'] = inst_df['DAYS_ENTRY_PAYMENT'] - inst_df['DAYS_INSTALMENT']
inst_df['DAYS_PAST_DUE'] = inst_df['DAYS_PAST_DUE'].apply(lambda x: x if x > 0 else 0)

# Days before due (no negative values)
inst_df['DAYS_BEFORE_DUE'] = inst_df['DAYS_INSTALMENT'] - inst_df['DAYS_ENTRY_PAYMENT']
inst_df['DAYS_BEFORE_DUE'] = inst_df['DAYS_BEFORE_DUE'].apply(lambda x: x if x > 0 else 0)

display(inst_df.head())

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,PAYMENT_PERCENT,PAYMENT_DIFF,DAYS_PAST_DUE,DAYS_BEFORE_DUE
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,1.000000,0.000,0.0,7.0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,1.000000,0.000,0.0,0.0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,1.000000,0.000,0.0,0.0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,1.000000,0.000,0.0,8.0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,0.997942,4.455,17.0,0.0


In [44]:
# Columns aggregations
inst_aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DAYS_PAST_DUE': ['max', 'mean', 'sum'],
    'DAYS_BEFORE_DUE': ['max', 'mean', 'sum'],
    'PAYMENT_PERCENT': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}
print(inst_aggregations)

{'NUM_INSTALMENT_VERSION': ['nunique'], 'DAYS_PAST_DUE': ['max', 'mean', 'sum'], 'DAYS_BEFORE_DUE': ['max', 'mean', 'sum'], 'PAYMENT_PERCENT': ['max', 'mean', 'sum', 'var'], 'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'], 'AMT_INSTALMENT': ['max', 'mean', 'sum'], 'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'], 'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']}


In [45]:
# Group by ID
inst_grouped = inst_df.groupby(ID).agg(inst_aggregations)

# Rename columns
inst_grouped.columns = rename_grouped_index(inst_grouped, prefix='INSTAL_')

display(inst_grouped.head())

,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DAYS_PAST_DUE_MAX,INSTAL_DAYS_PAST_DUE_MEAN,INSTAL_DAYS_PAST_DUE_SUM,INSTAL_DAYS_BEFORE_DUE_MAX,INSTAL_DAYS_BEFORE_DUE_MEAN,INSTAL_DAYS_BEFORE_DUE_SUM,INSTAL_PAYMENT_PERCENT_MAX,INSTAL_PAYMENT_PERCENT_MEAN,INSTAL_PAYMENT_PERCENT_SUM,...,INSTAL_AMT_INSTALMENT_MAX,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,2,11.0,1.571429,11.0,36.0,8.857143,62.0,1.0,1.0,7.0,...,17397.900,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,-1628.0,-2195.000000,-15365.0
100002,2,0.0,0.000000,0.0,31.0,20.421053,388.0,1.0,1.0,19.0,...,53093.745,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0
100003,2,0.0,0.000000,0.0,14.0,7.160000,179.0,1.0,1.0,25.0,...,560835.360,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0
100004,2,0.0,0.000000,0.0,11.0,7.666667,23.0,1.0,1.0,3.0,...,10573.965,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0
100005,2,1.0,0.111111,1.0,37.0,23.666667,213.0,1.0,1.0,9.0,...,17656.245,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,-470.0,-609.555556,-5486.0


In [46]:
# Count installments accounts
inst_grouped['INSTAL_COUNT'] = inst_df.groupby(ID).size()

In [47]:
# Merge to application dataframe
merged_df = merged_df.join(inst_grouped, how='left', on=ID)

### Credit card balance

* One hot encode categorical variables
* Add one feature : CC_COUNT (credit cards accounts)
* Merge credit_card_balance to application_train by ID

In [48]:
# Display dataframe
display(cc_df.head())

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [49]:
# One hot encode categorical features
cc_df, cc_encoded_cols = one_hot_encoding(cc_df, nan_as_category=True)
display(cc_df.head())

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_nan
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0,0,True,False,False,False,False,False,False,False
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,0,0,True,False,False,False,False,False,False,False
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,0,0,True,False,False,False,False,False,False,False
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,0,0,True,False,False,False,False,False,False,False
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,0,0,True,False,False,False,False,False,False,False


In [50]:
# Drop prev ID
cc_df = cc_df.drop(columns='SK_ID_PREV')

# Group by ID
cc_grouped = cc_df.groupby(ID).agg(['min', 'max', 'mean', 'sum', 'var'])

# Rename columns
cc_grouped.columns = rename_grouped_index(cc_grouped, prefix='CC_')

display(cc_grouped.head())

,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_MONTHS_BALANCE_VAR,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,CC_AMT_BALANCE_VAR,...,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-6,-1,-3.5,-21,3.5,0.0,0.00,0.000000,0.000,0.000000e+00,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100011,-75,-2,-38.5,-2849,462.5,0.0,189000.00,54482.111149,4031676.225,4.641321e+09,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100013,-96,-1,-48.5,-4656,776.0,0.0,161420.22,18159.919219,1743352.245,1.869473e+09,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100021,-18,-2,-10.0,-170,25.5,0.0,0.00,0.000000,0.000,0.000000e+00,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100023,-11,-4,-7.5,-60,6.0,0.0,0.00,0.000000,0.000,0.000000e+00,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0


In [51]:
# Count credit card lines
cc_grouped['CC_COUNT'] = cc_df.groupby(ID).size()

In [52]:
# Merge to application dataframe
merged_df = merged_df.join(cc_grouped, how='left', on=ID)
display(merged_df.head())

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002.0,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003.0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004.0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006.0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,6.0
4,100007.0,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Correlations

In [53]:
# Create correlation matrix
corr = merged_df.drop(columns=[ID, TARGET]).corr().abs()

In [54]:
# select upper traingle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
display(upper)

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
NAME_CONTRACT_TYPE,NaN,0.004196,0.063097,0.028738,0.004814,0.200187,0.228264,0.166609,0.023638,0.079442,...,0.001804,0.002206,0.003267,0.002945,NaN,NaN,NaN,NaN,NaN,0.009470
FLAG_OWN_CAR,NaN,NaN,0.002093,0.103036,0.089027,0.116992,0.143513,0.121241,0.039089,0.130519,...,0.003321,0.002312,0.004528,0.002301,NaN,NaN,NaN,NaN,NaN,0.041347
FLAG_OWN_REALTY,NaN,NaN,NaN,0.001880,0.004258,0.034738,0.001626,0.042336,0.018161,0.117175,...,0.005238,0.002344,0.005463,0.002781,NaN,NaN,NaN,NaN,NaN,0.050258
CNT_CHILDREN,NaN,NaN,NaN,NaN,0.014067,0.005988,0.025988,0.002243,0.024205,0.329198,...,0.003153,0.005118,0.006415,0.007471,NaN,NaN,NaN,NaN,NaN,0.036946
AMT_INCOME_TOTAL,NaN,NaN,NaN,NaN,NaN,0.166590,0.204391,0.169445,0.081087,0.028313,...,0.004592,0.005854,0.002006,0.009036,NaN,NaN,NaN,NaN,NaN,0.041567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CC_NAME_CONTRACT_STATUS_nan_MAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC_NAME_CONTRACT_STATUS_nan_MEAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC_NAME_CONTRACT_STATUS_nan_SUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC_NAME_CONTRACT_STATUS_nan_VAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.98)]
print(to_drop)
print(len(to_drop))

['AMT_GOODS_PRICE', 'YEARS_BUILD_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'OBS_60_CNT_SOCIAL_CIRCLE', 'CODE_GENDER_M', 'NAME_INCOME_TYPE_Pensioner', 'ORGANIZATION_TYPE_XNA', 'BURO_MONTHS_BALANCE_MIN_MIN', 'BURO_CREDIT_ACTIVE_Closed_MEAN', 'PREV_AMT_CREDIT_MAX', 'PREV_AMT_GOODS_PRICE_MAX', 'PREV_FLAG_LAST_APPL_PER_CONTRACT_Y_MEAN', 'PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN', 'PREV_NAME_PAYMENT_TYPE_XNA_MEAN', 'PREV_CODE_REJECT_REASON_CLIENT_MEAN', 'PREV_NAME_GOODS_CATEGORY_XNA_MEAN', 'PREV_PRODUCT_COMBINATION_nan_MEAN', 'POS_COUNT', 'INSTAL_PAYMENT_PERCENT_SUM', 'INSTAL_AMT_PAYMENT_MAX', 'INSTAL_AMT_PAYMENT_SUM', 'CC_MONTHS_BALANCE_MEAN', 'CC_MONTHS_BALANCE_SUM', 'CC_MONTHS_BALANCE_VAR', 'CC_AMT_PAYM

In [56]:
# Drop these features
merged_df = merged_df.drop(columns=to_drop)

## Final dataframe


In [57]:
# Final dataframe
display(merged_df)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
0,100002.0,0,0,1,0,202500.0,406597.5,24700.5,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003.0,0,0,0,0,270000.0,1293502.5,35698.5,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004.0,1,1,1,0,67500.0,135000.0,6750.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006.0,0,0,1,0,135000.0,312682.5,29686.5,0.008019,-19005,...,False,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0
4,100007.0,0,0,1,0,121500.0,513000.0,21865.5,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221.0,0,0,1,0,121500.0,412560.0,17473.5,0.002042,-19970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48740,456222.0,0,0,0,2,157500.0,622413.0,31909.5,0.035792,-11186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223.0,0,1,1,1,202500.0,315000.0,33205.5,0.026392,-15922,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48742,456224.0,0,0,0,0,225000.0,450000.0,25128.0,0.018850,-13968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Check missing values
nan_values = merged_df.isna().sum().sum()
print("Missing values = {} ({:.2%})".format(nan_values, nan_values/merged_df.size))

# num col only
numeric_df = merged_df.select_dtypes(include=[np.number])

# Check infinite values
inf_values = np.isinf(numeric_df).values.sum()
print("Infinite values = {} ({:.2%})".format(inf_values, inf_values/numeric_df.size))

Missing values = 42499649 (19.65%)
Infinite values = 40 (0.00%)


In [59]:
# Replace inf values
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)

## Export

In [62]:
# Split dataset
merged_test_df = merged_df[merged_df[ID].isin(test_ids)]
merged_train_df = merged_df[~merged_df[ID].isin(test_ids)]
merged_full_df = merged_df[~merged_df[ID].isin(test_ids)]

display("Test : ",merged_test_df.head())
display("Train : ",merged_train_df.head())
display("Full : ",merged_full_df.head())

'Test : '

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
0,100001.0,0,0,1,0,135000.0,568800.0,20560.5,0.018850,-19241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005.0,0,0,1,0,99000.0,222768.0,17370.0,0.035792,-18064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100013.0,0,1,1,0,202500.0,663264.0,69777.0,0.019101,-20038,...,False,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0
3,100028.0,0,0,1,2,315000.0,1575000.0,49018.5,0.026392,-13976,...,False,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0
4,100038.0,0,1,0,1,180000.0,625500.0,32067.0,0.010032,-13040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


'Train : '

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
0,100002.0,0,0,1,0,202500.0,406597.5,24700.5,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003.0,0,0,0,0,270000.0,1293502.5,35698.5,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004.0,1,1,1,0,67500.0,135000.0,6750.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006.0,0,0,1,0,135000.0,312682.5,29686.5,0.008019,-19005,...,False,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0
4,100007.0,0,0,1,0,121500.0,513000.0,21865.5,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


'Full : '

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
0,100002.0,0,0,1,0,202500.0,406597.5,24700.5,0.018801,-9461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003.0,0,0,0,0,270000.0,1293502.5,35698.5,0.003541,-16765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004.0,1,1,1,0,67500.0,135000.0,6750.0,0.010032,-19046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006.0,0,0,1,0,135000.0,312682.5,29686.5,0.008019,-19005,...,False,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0
4,100007.0,0,0,1,0,121500.0,513000.0,21865.5,0.028663,-19932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# Export datasets without column ID
merged_test_df.drop(columns=[ID, TARGET]).to_csv('../Sources/test_feature_engineering.csv')
merged_train_df.drop(columns=ID).to_csv('../Sources/train_feature_engineering.csv')
merged_full_df.to_csv('../Sources/full_features_engineering.csv')